In [23]:
import json
from py2neo import Graph
from py2neo.data import Node, Relationship

graph = Graph("bolt://neo4j:7687", auth=('neo4j','science'))
graph.delete_all()

In [24]:
# Constrain one id per Quanta
print("Creating uniqueness constraint (and index) on Quanta id's...", end=" ", flush=True)
query = """CREATE CONSTRAINT ON (n:Quanta) ASSERT n.id IS UNIQUE;"""
graph.run(query).evaluate()
print("Done.")

Creating uniqueness constraint (and index) on Quanta id's... Done.


In [25]:
# Create nodes 

local_data_dir = '/tmp/data/test/'
neo4j_data_dir = '/import/test/'

import glob, os
for _, _, files in os.walk(local_data_dir):
    for file in files:
        if file.endswith('.txt'):
            
#             # Normal query
#             query = """
#             CALL apoc.load.json("file://{}{}") 
#             YIELD value AS q
#             UNWIND q.id AS id
#             MERGE (i:Quanta {{id:q.id}})
#             ON CREATE SET i.abstract=q.abstract, i.refs=q.references, i.title=q.title, 
#                           i.fos=q.fos, i.url=q.url, i.lang=q.lang
#         """.format(neo4j_data_dir, file)
            
            # Iterative query (more efficient)
            print("Importing {}...".format(file), end=" ", flush=True)
            query = """
            CALL apoc.periodic.iterate(
            "CALL apoc.load.json('file://{}{}') 
                YIELD value AS q UNWIND q.id AS id RETURN q", 
            "MERGE (i:Quanta {{id:q.id}})
                ON CREATE SET i.refs=q.references, 
                i.title=q.title, i.fos=q.fos, i.url=q.url, i.lang=q.lang", 
            {{batchSize:10000, iterateList:true, parallel:false}});
            """.format(neo4j_data_dir, file)
#             print(query)
            graph.run(query).evaluate()
            print("Done.")

Importing mag_test_1.txt... Done.
Importing mag_10entries.txt... 

ClientError: Procedure Call Failed: Failed to invoke procedure `apoc.periodic.iterate`: Caused by: org.codehaus.jackson.JsonParseException: Unexpected character ('<' (code 60)): expected a valid value (number, String, array, object, 'true', 'false' or 'null')
 at [Source: apoc.export.util.CountingInputStream@6daa984d; line: 7, column: 2]

In [78]:
# Add all citations as relationships between Quanta

# # Simple but slow
# query = """
# MATCH (a:Quanta), (b:Quanta)
# WHERE a.id IN b.refs
# CREATE (b)-[:CITES]->(a)
# """

# # Faster but more complex (not benchmarked though)
# query = """
# CALL apoc.periodic.iterate(
#    "MATCH (a:Quanta), (a2:Quanta) WHERE a.id IN a2.refs
#     WITH a, COLLECT(a2) as b
#     RETURN a, b",
#    "UNWIND b AS a2
#     CREATE (a2)-[:CITES]->(a)",
#     {batchSize:5000, parallel:true,iterateList:true}
# """

# # Faster but simple (also not benchmarked)
# query = """
# CALL apoc.periodic.iterate(
#    "MATCH (a:Quanta), (b:Quanta) WHERE ID(a) < ID(b) AND a.id IN b.refs RETURN a, b",
#    "CREATE (b)-[:CITES]->(a)",
#     {batchSize:10000, parallel:true,iterateList:true})
# """

# Take advantage of indexing performed by constraints
print("Adding citations...", end=" ", flush=True)
query = """
MATCH (b:Node)
UNWIND b.refs AS ref
MATCH (a:Node)
WHERE a.id = ref
CREATE (b)-[:CITES]->(a);
"""
graph.run(query).evaluate()
print("Done.")
